In [8]:
from collections import Counter
import pandas as pd

In [3]:
file_name = "9M-5-[1-2001]-[a]-lcm.out"

# Product adoption & Loyalty

In [7]:
links = pd.read_csv(f"../../plots/links/{file_name}")
links = links.drop(["user_id"],axis=1)
groups = pd.read_csv(f"../../plots/groups/{file_name}",index_col=0)
groups = groups.drop(['depth',"a","size","support",],axis=1)
df = links.merge(groups.reset_index(),left_on="source",right_on='index').merge(groups.reset_index(),left_on="target",right_on='index')

In [18]:
before_promo_period = groups.period.min()
after_promo_period = groups.period.max()
promo_period = list(set(groups.period.unique()).difference((before_promo_period,after_promo_period)))[0]

promo_items = pd.read_csv("../../datasets/Total/items.csv",encoding="latin",sep=";")
promo_list_items = promo_items.ARTICLE_ID.tolist()

def promotion_query(x,query_name):
    if x["period_x"]!= before_promo_period or x["period_y"]!= promo_period:
        return set()
    p1 = x["itemsets_x"].split()
    p2 = set(x["itemsets_y"].split())
    if query_name =="adoption":
        return p2.difference(p1)
    if query_name =="loyalty":
        return p2.intersection(p1)
df["adopted_products"] = df.apply(lambda x: promotion_query(x,"adoption"),axis=1)
df["loyalty_products"] = df.apply(lambda x: promotion_query(x,"loyalty"),axis=1)
df


,source,target,index_x,user_ids_x,itemsets_x,period_x,property_values_x,itemsets_name_x,index_y,user_ids_y,itemsets_y,period_y,property_values_y,itemsets_name_y,adopted_products,loyalty_products
0,3,785,3,[ 171222 331511 337743 387123 409260 4383...,3450013,2018-09-01,1,CR SANDWHICH OCEANIQUE (new); CR SANDWHICH OC,785,[ 145422 264552 277744 296455 305419 3225...,3461025,2018-12-01,1,CR EVIAN PET 50CL; CR EVIAN PET 50,{3461025},{}
1,9,785,9,[ 280999 353744 575459 581812 1108766],4159045:2850269,2018-09-01,1,CR COOKIE CHOCOLAT LAIT 103 GR; CR COOKIE CHOCO,785,[ 145422 264552 277744 296455 305419 3225...,3461025,2018-12-01,1,CR EVIAN PET 50CL; CR EVIAN PET 50,{3461025},{}
2,11,785,11,[ 104790 204037 575459 903174 1166138],4159045:3450014,2018-09-01,1,CR COOKIE CHOCOLAT LAIT 103 GR; CR COOKIE CHOCO,785,[ 145422 264552 277744 296455 305419 3225...,3461025,2018-12-01,1,CR EVIAN PET 50CL; CR EVIAN PET 50,{3461025},{}
3,20,785,20,[ 267648 355952 400827 425686 575459 7250...,3450042,2018-09-01,1,Cro Hot Dog Bacon Cheese; Cro Hot Dog Bac,785,[ 145422 264552 277744 296455 305419 3225...,3461025,2018-12-01,1,CR EVIAN PET 50CL; CR EVIAN PET 50,{3461025},{}
4,21,785,21,[ 104790 204037 280999 314494 331511 3537...,4159045,2018-09-01,1,CR COOKIE CHOCOLAT LAIT 103 GR; CR COOKIE CHOCO,785,[ 145422 264552 277744 296455 305419 3225...,3461025,2018-12-01,1,CR EVIAN PET 50CL; CR EVIAN PET 50,{3461025},{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665,790,1524,790,[ 127376 193219 204060 208883 255796 2890...,7070091000701,2018-12-01,1,TOTAL ADBLUE 10 L; TOTAL ADBLUE 10L; TOTAL ADB...,1524,[142784 193219 220023 385430 482729 556219 597...,7070091000701,2019-02-01,1,TOTAL ADBLUE 10 L; TOTAL ADBLUE 10L; TOTAL ADB...,{},{}
666,791,834,791,[ 336115 559661 560180 579539 810351 10116...,3460537800028,2018-12-01,1,TOTAL ADBLUE 10 L; TOTAL ADBLUE 10L; TOTAL ADB...,834,[ 230124 328201 337592 533301 547651 5795...,100,2019-02-01,1,NaN,{},{}
667,791,1507,791,[ 336115 559661 560180 579539 810351 10116...,3460537800028,2018-12-01,1,TOTAL ADBLUE 10 L; TOTAL ADBLUE 10L; TOTAL ADB...,1507,[ 214946 254801 334074 467853 579539 7840...,106,2019-02-01,1,NaN,{},{}
668,791,1529,791,[ 336115 559661 560180 579539 810351 10116...,3460537800028,2018-12-01,1,TOTAL ADBLUE 10 L; TOTAL ADBLUE 10L; TOTAL ADB...,1529,[375791 516890 579539 816656 897203],3460537800028,2019-02-01,1,TOTAL ADBLUE 10 L; TOTAL ADBLUE 10L; TOTAL ADB...,{},{}


In [17]:
for i in df.adopted_products.values.tolist():
    if i ==set():
        continue
    res+= list(i)
Counter(res)

Counter({'3461025': 100,
         '3701239300011': 72,
         '3217770310902': 192,
         '3068320099651': 180,
         '3460537800028': 2,
         '7070091000701': 2,
         '3425901028170': 30,
         '3425901018140': 60,
         '3425901018720': 28,
         '4042865227723': 6,
         '3217770310513': 2})